In [1]:
import pycisTopic
pycisTopic.__version__

'0.1.dev300+g7494158'

In [2]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore') 

In [3]:
import pickle
import pandas as pd

In [4]:
import os
wdir = '/lustre1/project/stg_00002/lcb/fderop/data/20211024_hca_benchmark_libds_merged//'
os.chdir( wdir )

# read CTOs

In [5]:
f_cto_dir = 'pycistopic_consensus_peaks/cistopic_objs__mergedconsensus/'

cistopic_obj_dict = {}
for key in ['libds_merged']:
    f_cto = os.path.join(wdir, f_cto_dir, key + '__cistopic_obj_mergedconsensus_metadata_annotated.pkl')
    if(os.path.isfile(f_cto)):
        with open(f_cto, 'rb') as f:
            cistopic_obj_dict[key] = pickle.load(f)
        print(f"Loaded filtered cistopic object {key}")
    else:
        print(f"file {f_cto} doesn't exist")

Loaded filtered cistopic object libds_merged


### Run models for quick visualize/clustering

In [6]:
from pycisTopic.lda_models import run_cgs_models_mallet

#### mallet models

In [7]:
# set the memory limit for mallet (1GB default is not enough)
os.environ['MALLET_MEMORY'] = '100G'

In [8]:
f_mod_dir = 'pycistopic_consensus_peaks/models__consensus__mallet_alltopics'
if not os.path.exists(os.path.join(wdir, f_mod_dir)):
    os.makedirs(os.path.join(wdir, f_mod_dir))

f_mod_tmpdir = '/scratch/leuven/330/vsc33042/tmp/mallet_cellds_merged'
if not os.path.exists(f_mod_tmpdir):
    os.makedirs(f_mod_tmpdir)

save_path = '/scratch/leuven/330/vsc33042/tmp/mallet_cellds_merged/models'
if not os.path.exists(os.path.join(save_path)):
    os.makedirs(os.path.join(save_path))    
    
n_topics = n_topics = [2, 5, 10, 20, 30, 35, 40, 45, 50, 60, 70, 80, 90, 100]
# n_topics_merged = [2, 5, 10, 20, 30, 35, 40, 45, 50, 60, 70, 80, 90, 100]
n_iter = 500

In [9]:
# mallet implementation
models_dict = {}
for key in ['libds_merged']:
    f_mod = os.path.join(wdir, f_mod_dir, key + '__models_' + str(n_iter) + '_iter.pkl')
    if os.path.isfile(f_mod):
        print(f"Loading {f_mod}")
        with open(f_mod, 'rb') as f:
            models_dict[key] = pickle.load(f)
    else:
        print(f"Running {key}")
        model = run_cgs_models_mallet(
                            'mallet',
                            cistopic_obj_dict[key],
                            n_topics=n_topics_merged if key=='merged' else n_topics ,
                            n_cpu=24,
                            n_iter=n_iter,
                            random_state=555,
                            alpha=50,
                            alpha_by_topic=True,
                            eta=0.1,
                            save_path=save_path,
                            eta_by_topic=False,
                            tmp_path=os.path.join(f_mod_tmpdir, key + '_')
                            )
        # Save
        with open(f_mod, 'wb') as f:
            pickle.dump(model, f)
        print(f"Finished {key}")
        models_dict[key] = model

Running libds_merged
2021-10-28 10:25:11,462 cisTopic     INFO     Formatting input to corpus


2021-10-28 10:25:20,397 gensim.corpora.dictionary INFO     adding document #0 to Dictionary(0 unique tokens: [])


2021-10-28 10:26:12,519 gensim.corpora.dictionary INFO     adding document #10000 to Dictionary(0 unique tokens: [])


2021-10-28 10:27:07,425 gensim.corpora.dictionary INFO     adding document #20000 to Dictionary(0 unique tokens: [])


2021-10-28 10:28:06,627 gensim.corpora.dictionary INFO     adding document #30000 to Dictionary(0 unique tokens: [])


...

2021-10-29 15:50:49,851 LDAMalletWrapper INFO     Training MALLET LDA with mallet train-topics --input /scratch/leuven/330/vsc33042/tmp/mallet_cellds_merged/libds_merged_corpus.mallet --num-topics 100  --alpha 50 --beta 0.1 --optimize-interval 0 --num-threads 24 --output-state /scratch/leuven/330/vsc33042/tmp/mallet_cellds_merged/libds_merged_11fe11_state.mallet.gz --output-doc-topics /scratch/leuven/330/vsc33042/tmp/mallet_cellds_merged/libds_merged_11fe11_doctopics.txt --output-topic-keys /scratch/leuven/330/vsc33042/tmp/mallet_cellds_merged/libds_merged_11fe11_topickeys.txt --num-iterations 500 --inferencer-filename /scratch/leuven/330/vsc33042/tmp/mallet_cellds_merged/libds_merged_11fe11_inferencer.mallet --doc-topics-threshold 0.0  --random-seed 555


2021-10-29 18:07:37,526 LDAMalletWrapper INFO     loading assigned topics from /scratch/leuven/330/vsc33042/tmp/mallet_cellds_merged/libds_merged_11fe11_state.mallet.gz


2021-10-29 18:21:13,707 cisTopic     INFO     Model with 100 topics done!


2021-10-29 18:21:13,708 cisTopic     INFO     Saving model with 100 topics at /scratch/leuven/330/vsc33042/tmp/mallet_cellds_merged/models


Finished libds_merged
